### Task 1: Determining Hidden Markov Model (HMM)

##### a. Write down the transition matrix $\mathbf{A}$ and the emission matrix $\mathbf{B}$


State Transition matrix:
$$
\mathbf{A} = 
\begin{bmatrix}
0.95 & 0.05 \\
0.1 & 0.9
\end{bmatrix}
$$

Emission matrix:
$$
\mathbf{B} = 
\begin{bmatrix}
1/6 & 1/6 & 1/6 & 1/6 & 1/6 & 1/6 \\
1/10 & 1/10 & 1/10 & 1/10 & 1/10 & 1/2
\end{bmatrix}
$$

##### b. For the observation sequence $\mathbf{O}=(1,6,6)$, compute the probabilities of the hidden sequences (b1) $\mathbf{Z}=(F,L,L)$, and (b2) $\mathbf{Z}=(F,F,F)$, i.e. $P(\mathbf{X}, \mathbf{Z}|\mathbf{\theta})$. Which is higher?

This is the decoder, which is probability of hidden states given model and observations.

$\underline{b1 - Z(F,L,L)}$: 

Probability of the hidden sequence (F,L,L): 
Start in F, transition to L, stay in L

$P(\mathbf{Z|\theta}$) = 0.5 * 0.05 * 0.9 = 0.0225

Probability of observation sequence (1,6,6) given (F,L,L) is:

$P(\mathbf{O}|\mathbf{Z}, \theta)$ = 1/6 * 1/2 * 1/2 = 0.041667

$P(\mathbf{\mathbf{O},\mathbf{Z}|\theta})$ = 0.0225 * 0.041667 = 0.0009375

Slides:

$P(\mathbf{O}|\mathbf{Z}, \mathbf{\theta})$ = 0.5 * 1/6 * 0.05 * 1/2 * 0.9 * 1/2 = 0.0009375



$\underline{b2 - Z(F,F,F)}$:

Probability of the hidden sequence (F,F,F):
Start in F, stay in F, stay in F

$P(\mathbf{Z|\theta}$) = 0.5 * 0.95 * 0.95 = 0.45125

Probability of observation sequence (1,6,6) given (F,F,F):

$P(\mathbf{O}|\mathbf{Z}, \theta)$ = 1/6 * 1/6 * 1/6 = 0.0046296

$P(\mathbf{\mathbf{O},\mathbf{Z}|\theta})$ = 0.45125 * 0.0046296 = 0.002089 


b2 is the most likely hidden state of the 2

##### c. How can the probability of a sequence given a model $P(\mathbf{X} | \mathbf{\theta})$ be computed? Give the equation in detail, i.e. how all needed $\mathbf{Z}$ look like.

This is the equation for evaluting the model $P(\mathbf{X}|\mathbf{\theta})$.
For this we use forward algorithm to recursively calculate partial-sequence probabilities.

Hence we need to compute the sum of the partial-sequence probabilities that can lead to this sequence. i.e. find all hidden statees that can lead to this sequence and their corresponding probabilities.

Given the example above (1,6,6), we need to find all hidden states that can lead to this:

(F,F,F), (F,F,L), (F,L,F), (F,L,L), (L,L,L), (L,L,F), (L,F,L), (L,F,F)

and for each of these, we need to compute the probability of this hidden state to occur times the probability of the hidden state resulting in the sequence observed:

With (F,F,F) as example:

$P(\mathbf{X}|(F,F,F))$ 0.5 * 0.95 * 0.95 for the state times 1/6 * 1/6 * 1/6 for the sequence observed = 0.002089

This should be summed with all the other combinations

$P(\mathbf{x}|\mathbf{\theta})$ = $\sum_{i=1}^8(P(x,Z_i|\theta))$

##### d. Implement a function to compute $P(\mathbf{X}, \mathbf{Z} | \mathbf{\theta})$

$$
P(\mathbf{X}, \mathbf{Z}|\mathbf{\theta}) = P(\mathbf{X}|\mathbf{\theta}) * P(\mathbf{Z}|\mathbf{\theta})
$$

In [3]:
#Implement function for forward algorithm

def score_sequences(sequences,initial_probs,transition_probs,emission_probs,obs):
    
    best_score = -1
    best_sequence = None
    
    sequence_scores = []
    for seq in sequences:
        total_score = 1
        total_score_breakdown = []
        first = True
        for i in range(len(seq)):
            state_score = 1
            # compute transitition probability score
            if first == True:
                state_score *= initial_probs[seq[i]]
                # reset first flag
                first = False
            else:  
                state_score *= transition_probs[seq[i] + "|" + seq[i-1]]
            # add to emission probability score
            state_score *= emission_probs[obs[i] + "|" + seq[i]]
            # update the total score
            #print state_score
            total_score_breakdown.append(state_score)
            total_score *= state_score
            
        sequence_scores.append(total_score)
        
    return sequence_scores

##### e. Implement the Forward-Algorithm. Compute 6.2(c) using Python code, i.e. define a function, which:
- receives as input an observation sequence $\mathbf{X}$, and
- provides the probability of on the observation sequence $P(\mathbf{X} | \mathbf{theta})$ as output.
Hint: consider how do you get all needed combinations of th hidden state sequences? Then start with a fixed observation length of your choice, e.g. 3, then expand to arbitrary lengths

In [4]:
# given states - what are the possible combinations
# total number of combinations is (number of possible states)^(sequence length)
def generate_sequence(states,sequence_length):
    
    all_sequences = []
    nodes = []
    
    depth = sequence_length
    
    def gen_seq_recur(states,nodes,depth):
        if depth == 0:
            #print nodes
            all_sequences.append(nodes)
        else:
            for state in states:
                temp_nodes = list(nodes)
                temp_nodes.append(state)
                gen_seq_recur(states,temp_nodes,depth-1)
    
    gen_seq_recur(states,[],depth)
                
    return all_sequences



# pretty printing our  distributions
import pandas as pd
from tabulate import tabulate
def pretty_print_probs(distribs):
    
    rows = set()
    cols = set()
    for val in distribs.keys():
        temp = val.split("|")
        rows.add(temp[0])
        cols.add(temp[1])
        
    rows = list(rows)
    cols = list(cols)
    df = []
    for i in range(len(rows)):
        temp = []
        for j in range(len(cols)):
            temp.append(distribs[rows[i]+"|"+cols[j]])
            
        df.append(temp)
        
    I = pd.Index(rows, name="rows")
    C = pd.Index(cols, name="cols")
    df = pd.DataFrame(data=df,index=I, columns=C)
    
    print(tabulate(df, headers='keys', tablefmt='psql'))
def initializeSequences(_obs):
    # Generate list of sequences
    
    seqLen = len(_obs)
    seqs = generate_sequence(states,seqLen)
    # Score sequences
    seq_scores = score_sequences(seqs,initial_probs,transition_probs,emission_probs,obs)
    
    return (seqLen,seqs,seq_scores)

# We can use a dictionary to capture our state transitions
# set of hidden states
states = ['F','L']
# set of observations
obs = ['1','6','6']
# set of hidden layers
hidden = ['F','F','F']
# initial state probability distribution (our priors)
initial_probs = {'F':0.5,'L':0.5}
# transition probabilities
transition_probs = {'F|F':0.95,'L|F':0.05,'F|L':0.1,'L|L':0.9}
# emission probabilities
emission_probs = {'1|F':1/6, '2|F':1/6, '3|F':1/6, '4|F':1/6, '5|F':1/6, '6|F':1/6, '1|L':1/10, '2|L':1/10, '3|L':1/10, '4|L':1/10, '5|L':1/10, '6|L':1/2}
# Generate list of sequence
sequence_length,sequences,sequence_scores = initializeSequences(obs)
# print results
print("Initial Distributions")
print(initial_probs)
print("\nTransition Probabilities")
pretty_print_probs(transition_probs)
print("\nEmission Probabilities")
pretty_print_probs(emission_probs)
print("\nScores")
# Display sequence scores
for i in range(len(sequences)):
    print("Sequence:%10s,Prob:%0.4f" % (sequences[i],sequence_scores[i]))

# Display prob of obs given sequence
print("\nScores with the hidden sequence")
for i in range(len(sequences)):
    if sequences[i] == hidden:
        print("Hidden Sequence:%10s, Yields prob:%0.4f" % (sequences[i],sequence_scores[i]))



Initial Distributions
{'F': 0.5, 'L': 0.5}

Transition Probabilities
+--------+------+-----+
| rows   |    F |   L |
|--------+------+-----|
| F      | 0.95 | 0.1 |
| L      | 0.05 | 0.9 |
+--------+------+-----+

Emission Probabilities
+--------+----------+-----+
|   rows |        F |   L |
|--------+----------+-----|
|      4 | 0.166667 | 0.1 |
|      2 | 0.166667 | 0.1 |
|      6 | 0.166667 | 0.5 |
|      1 | 0.166667 | 0.1 |
|      3 | 0.166667 | 0.1 |
|      5 | 0.166667 | 0.1 |
+--------+----------+-----+

Scores
Sequence:['F', 'F', 'F'],Prob:0.0021
Sequence:['F', 'F', 'L'],Prob:0.0003
Sequence:['F', 'L', 'F'],Prob:0.0000
Sequence:['F', 'L', 'L'],Prob:0.0009
Sequence:['L', 'F', 'F'],Prob:0.0001
Sequence:['L', 'F', 'L'],Prob:0.0000
Sequence:['L', 'L', 'F'],Prob:0.0004
Sequence:['L', 'L', 'L'],Prob:0.0101

Scores with the hidden sequence
Hidden Sequence:['F', 'F', 'F'], Yields prob:0.0021


### Task 2: Understanding Basic Recurrent Neural Networks (RNNs)

##### a. Retrace the data loading and preprocessing and take your time to clarify:
- How and why is the vocabulary dictionary built up?
    - It's built up on token level and depends on the frequency of each token
    - Each token can either be a word or a character
    - The vocabulary consists of vectors of 0 except for the index at the word: for one-hot encoding
    - self.token_freqs -> tokens and counts of tokens in most frequent order
    - self.uniq_tokens -> unique tokens with a frequence above som threshold
    - self.idx_to_tokens -> way of getting each token from index
    - self.tokens_to_idx -> way of getting index of each token
- What is the role of "num_steps"?
    - The length of sequences
    - We need this to standardize the inputs.
        - if not the same length, we use padding

##### b. Retrace the model (first the implementation from scratch, then the RNN sub-module of the framework) and familiarize yourself with the data structure of the network tensor: ('num_steps', 'batch_size', 'vocab_size')
- Why is it a good idea to fix these parameters in the model building?
    - The different inputs need the same model layout to compile and function
    - Enables in determining the other hyperparameters: weight matrix and bias vector.
- What is the effect of a small versus a large vocab_size?
    - Large vocab_size requires a lot more training data but could probably result in more fine tuned language in the end. For this data though, it quickly gets too large, so it's best to keep small for actually getting words.
    - Smaller vocab gives smaller computations. Larger gives much more computations. 
    - The dimensionality of the space gets higher with more vocabs.

##### c. Retrace the training steps and focus on the following questions:
- How and why are hidden states initialized during training batches (and how is this different to non-Recurrent NN training)?
    - state = net.begin_state(batch_size=X.shape[0], device=device)
    - Hidden states are initialized during training batches as they depent on the previous outputs predicted. We therefore need each training batch to have a clean start if we use random sequence.
    - It's not relevant for non-Recurrent NN training as the current prediction isn't dependent on the previous predictions. 
- Specifically: what would be the effect of initializing vs not initializing hidden states at the begining of each first time step of a sequence?
    - It simulates no dependency on the previous sequence.


##### d. Initialize the hidden states (init_rnn_states) instead of with zeros with other values (e.g. random values between 0 and 1).
- What is the effect?
    - Seems to get better values -> because they are closer than all at zero?
    - Starting it with random numbers forces the model to think this depends on something it has learned in the past.
- How does the training behave when we use no gradient clipping?
    - Didn't test but would probably get exploding gradients. The weights get huge.
- What is the effect on the training when using other values for num_steps: e.g. 15, 60, and 120?
    - No real difference. Same performance speed and outcomes
    - Small number of steps prevents the model from retaining a large memory.

Bonus: Perplexity is a way of counting the likelihood by counting. This is to prevent just looking at the probability, which would almost always be so small the computer would translate to 0, due to there being so many different combinations of words that can make up a sentence.

### Task 3: Understanding Gating in RNNs
Retrace the implementation of the model and:

##### a. Modify the example to run with the SimpleRNN instead of the GRU and note down your result as a baseline. How did the network perform (quality)?

Results:

##### b. Now run the model using GRU: what are the major effects in training (speed and convergence) and results (quality)?

Results

##### c. Modify the example to run with LSTM cells instead:
- What is the effect?
- Is the performance different thatn compared to using GRU?
    - Should be quicker trained
    - Better trained as it can retain information

##### d. Compare the parameters of the needed number for decent performance between GRU, LSTM and the baseline (the baseline/SimpleRNN might not converge)?

### Task 4: Understanding Word Embeddings

##### a. So, first of all, explore learned word embeddings with Word2Vec
- What is the difference in data and effect of Word2Vec All, Word2Vec 10k?
    - One of them as a lot more vectors (71k) compared to only 10k
    - Due to it mapping all words to vectors
    - word2Vec all - suggest/maps a lot more rare words 0 which can be both for better and worse
        - probably more nuanced suggestions, but can also be more confusing or not what we expect
- What are your most 1) intuitive and 2) counter-intuitive observations regarding the vector spaces?
    - Kg -> both maps a lot of weight measurring units
    - irrelevent (spelling mistake?) -> mapped to other useless stuff
        - Doesn't even exist in Word2Vec 10k


##### b. Next, reproduce how building embeddings from skip-grams work (again, focus on the questions below, otherwise you might drown in code and details)
- Refresh yourself and understand: how is the tokenisation realized?
- Try different embedding sizes. How do similar tokens compare? Test own token examples as well!
- Figure out and test different window sizes for the skip-grams. What is the impacto n quality (e.g. similarity of tokens) vs. effor (training speed and convergence)?